In [1]:
# venv: bouton_ana
import pandas as pd
import numpy as np
import shutil
import os
import imageio
from PIL import Image,ImageDraw,ImageFont
import random
import math

In [17]:
def saveswc(inswc,tofile,scale=1,retype=None,xshift=0,flip=False,header=None):
    swc=inswc.copy()
    if header is None:
        header=['##n','type','x','y','z','radius','parent']
    if retype is not None:
        swc['type']=retype
    if scale != 1 and scale >0:
        swc['x']/=scale
        swc['y']/=scale
        swc['z']/=scale
        if xshift:
            swc['x']+=xshift
        if flip:
            soma=swc[swc.parent<0].copy()
            soma=soma.iloc[0]
            sz=soma['z']
            if sz > 456/2:
                # print('flip')
                swc['z']= 456 - swc['z']
    swc.reset_index(inplace=True)
    if swc.shape[1] != len(header):
        print('Input SWC header not match!!!')
        header=swc.keys().to_list()
        header[0]='##'+header[0]
    swc.to_csv(tofile,sep=' ',index=0,header=header)
    # return swc

def get_rid(swc):
    '''
    Find root node.
    '''
    rnode=swc[((swc['parent']<0) & (swc['type']<=1))]
    if rnode.shape[0]<1:
        return -1
    return rnode.index[0]

def readswc(file,out_fea=None,fea_val_names=None,header=None,skiprows=3):
    swc=pd.read_csv(file,sep=' ',header=None,skiprows=skiprows)
    if header is None:
        header=['n','type','x','y','z','radius','parent']
        if swc.shape[1] >= 12:
            header+=['seg_id','level','mode','timestamp','teraflyindex']
            for i in np.arange(12,swc.shape[1]):
                if fea_val_names is not None:
                    header.append(fea_val_names[i-12])
                else:
                    header.append('fea_'+str(i-12))
    if len(header) == swc.shape[1]:
        swc.columns=header
    # print(swc)
    swc.set_index(['n'],drop=True,inplace=True)
    if out_fea is None:
        return swc
    else:
        return swc[out_fea]
def readswc_arbor(file,skiprows=1):
    header=['n','type','x','y','z','radius','parent','label']
    swc=pd.read_csv(file,sep=' ',header=None,skiprows=skiprows)
    if len(header) == swc.shape[1]:
        swc.columns=header
        # print(swc)
        swc.set_index(['n'],drop=True,inplace=True)
        return swc
    else:
        print(file,'read fail')
        return None

## For Web data

In [148]:
'''For soma morphometry
1. read soma morphometry
2. convert to marker
3. render in vtk and generate image
4. image seq to video
'''
somaf='TableS3_Soma_morphometry_1222.csv'
somadf=pd.read_csv(somaf)
soma_in_brain=somadf[['Soma ID', 'fMOST BrainID', 'Soma_X(CCFv3_1𝜇𝑚)', 'Soma_Y(CCFv3_1𝜇𝑚)', 'Soma_Z(CCFv3_1𝜇𝑚)','Registered  CCF Region']].copy()
head='##n,orderinfo,name,comment,z,x,y,pixmax,intensity,sdev,volsize,mass,f1,f2,f3,color_r,color_g,color_b'
#0,,18454,,3964.11,8454.241,4969.068,0,0,0,100,0,,,,125.0,250.0,196.0
for bid in somadf['fMOST BrainID'].value_counts().index.to_list():
    somas=soma_in_brain[soma_in_brain['fMOST BrainID']==bid].copy()
    region_sorted=somas['Registered  CCF Region'].value_counts().index.to_list()
    somas['r']=0
    somas['g']=0
    somas['b']=0
    somas['c']=-1
    for i,region in enumerate(region_sorted):
       somas.loc[somas['Registered  CCF Region']==region,'c']=i
       clist=random.choices(np.arange(255),k=3)
       somas.loc[somas['Registered  CCF Region']==region,'r']=clist[0]
       somas.loc[somas['Registered  CCF Region']==region,'g']=clist[1]
       somas.loc[somas['Registered  CCF Region']==region,'b']=clist[2]
    soma_markers=pd.DataFrame()
    for cell in somas.index:
       if somas.loc[cell,'c'] < 0:
          continue
       soma_markers.loc[cell,'##x']=int(somas.loc[cell,'Soma_X(CCFv3_1𝜇𝑚)']/25+20)
       soma_markers.loc[cell,'y']=int(somas.loc[cell,'Soma_Y(CCFv3_1𝜇𝑚)']/25)
       soma_markers.loc[cell,'z']=int(somas.loc[cell,'Soma_Z(CCFv3_1𝜇𝑚)']/25)
       soma_markers.loc[cell,'radius']=0
       soma_markers.loc[cell,'shape']=1
       soma_markers.loc[cell,'name']=somas.loc[cell,'Registered  CCF Region']
       soma_markers.loc[cell,'comment']=somas.loc[cell,'c']
       soma_markers.loc[cell,'color_r']=somas.loc[cell,'r']
       soma_markers.loc[cell,'color_g']=somas.loc[cell,'g']
       soma_markers.loc[cell,'color_b']=somas.loc[cell,'b']
    soma_markers.to_csv('.soma_markers/'+str(bid)+'.marker',index=False,float_format='%.0f')

In [5]:
# image sequence to video
image_src='soma_figs'
# 读取所有 PNG 图片
color=(255,0,0)
images = []
for file_name in sorted(os.listdir(image_src)):
    if file_name.endswith('.png'):
        fpath=os.path.join(image_src,file_name)
        fimage=Image.open(fpath).resize([600,600]).rotate(270)
        drawimage=ImageDraw.Draw(fimage)
        text="Brain : "+file_name.split('.')[0]
        font=ImageFont.truetype('Arial Unicode.ttf', size=50)
        # text_width, text_height = drawimage.textsize(text, font)
        x = 150
        y = 30
        drawimage.text((x, y), text, font=font, fill=color)
        # break
        images.append(fimage)
images=np.array(images)
# fimage
# 将图片转换为视频
fps = 1  # 每秒钟30帧
filepath='./soma.mp4'
with imageio.get_writer(filepath, fps=fps) as video:
    for image in images:
        frame = image
        video.append_data(frame)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 600) to (608, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[swscaler @ 0x7f8f3f557000] Warning: data is not aligned! This can lead to a speed loss


In [6]:
'''For local morphometry:
1. morphology by image id
2. vtk
3. to video
'''

image_src='local_figs'
# 读取所有 PNG 图片
color=(255,0,0)
images = []
for file_name in sorted(os.listdir(image_src)):
    if file_name.endswith('.png'):
        fpath=os.path.join(image_src,file_name)
        fimage=Image.open(fpath).resize([600,600]).rotate(270)
        drawimage=ImageDraw.Draw(fimage)
        text="Brain : "+file_name.split('.')[0]
        font=ImageFont.truetype('Arial Unicode.ttf', size=50)
        # text_width, text_height = drawimage.textsize(text, font)
        x = 150
        y = 30
        drawimage.text((x, y), text, font=font, fill=color)
        # break
        images.append(fimage)
images=np.array(images)
# fimage
# 将图片转换为视频
fps = 1  # 每秒钟30帧
filepath='./local.mp4'
with imageio.get_writer(filepath, fps=fps) as video:
    for image in images:
        frame = image
        video.append_data(frame)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 600) to (608, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[swscaler @ 0x7fc9f5257000] Warning: data is not aligned! This can lead to a speed loss


In [7]:
'''For full morphometry:
'''

image_src='full_figs'
# 读取所有 PNG 图片
color=(255,0,0)
images = []
for file_name in sorted(os.listdir(image_src)):
    if file_name.endswith('.png'):
        fpath=os.path.join(image_src,file_name)
        fimage=Image.open(fpath).resize([600,600]).rotate(270)
        drawimage=ImageDraw.Draw(fimage)
        text="Brain : "+file_name.split('.')[0]
        font=ImageFont.truetype('Arial Unicode.ttf', size=50)
        # text_width, text_height = drawimage.textsize(text, font)
        x = 150
        y = 30
        drawimage.text((x, y), text, font=font, fill=color)
        # break
        images.append(fimage)
images=np.array(images)
# fimage
# 将图片转换为视频
fps = 1  # 每秒钟30帧
filepath='./full.mp4'
with imageio.get_writer(filepath, fps=fps) as video:
    for image in images:
        frame = image
        video.append_data(frame)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 600) to (608, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[swscaler @ 0x7f9ef0a57000] Warning: data is not aligned! This can lead to a speed loss


In [57]:
'''For bouton morphometry: compression
'''
all1876_feas_file='R1876_feas.csv'
all_1876_feas=pd.read_csv(all1876_feas_file,index_col=['name'])
image_src='bouton_figs'
toimage_src='bouton_figs3'
# 读取所有 PNG 图片
color=(255,0,0)
for region in all_1876_feas['soma_region'].value_counts().keys().tolist():
    tregion=all_1876_feas[all_1876_feas['soma_region']==region].copy()
    for cell in tregion.index:
        file_name=cell+'.png'
        fpath=os.path.join(image_src,file_name)
        tofig=os.path.join(toimage_src,file_name)
        if os.path.exists(fpath) and not os.path.exists(tofig):
            fimage=Image.open(fpath).resize([600,600])
            drawimage=ImageDraw.Draw(fimage)
            text=region
            font=ImageFont.truetype('Arial Unicode.ttf', size=38)
            drawimage.text((10, 50), text, font=font, fill=color)
            fimage.save(tofig,quality=1)


In [56]:
'''For bouton morphometry:
'''
all1876_feas_file='R1876_feas.csv'
all_1876_feas=pd.read_csv(all1876_feas_file,index_col=['name'])
image_src='bouton_figs2'
# 读取所有 PNG 图片
color=(255,0,0)
images = []
for region in all_1876_feas['soma_region'].value_counts().keys().tolist():
    tregion=all_1876_feas[all_1876_feas['soma_region']==region].copy()
    for cell in tregion.index:
        file_name=cell+'.png'
        fpath=os.path.join(image_src,file_name)
        if os.path.exists(fpath):
            fimage=Image.open(fpath)
            # drawimage=ImageDraw.Draw(fimage)
            # text=region
            # font=ImageFont.truetype('Arial Unicode.ttf', size=38)
            # # text_width, text_height = drawimage.textsize(text, font)
            # x = 10
            # y = 50
            # drawimage.text((x, y), text, font=font, fill=color)
            # break
            images.append(fimage)

images=np.array(images)
# fimage
fps = 3
filepath='./bouton_mst.mp4'
with imageio.get_writer(filepath, fps=fps) as video:
    for image in images:
        frame = image
        video.append_data(frame)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 600) to (608, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


In [6]:
# For motif
motifdir='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/SD/release/motif/CCFv3'
todir='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/SD/release/motif/CCFv3_25um_reorg'
if not os.path.exists(todir):
    os.mkdir(todir)
for root,dirs,files in os.walk(motifdir):
    for mfile in files:
        if os.path.splitext(mfile)[1] != '.swc':
            continue
        if mfile.split('_')[0] == 'unk':
            continue
        file_in=os.path.join(root,mfile)
        ptype=mfile.split('-')[0]
        ctype=mfile[len(ptype):-1].split('_')[0]
        topath=os.path.join(todir,ptype+ctype)
        if not os.path.exists(topath):
            os.mkdir(topath)
        # break
        swc=readswc(file=file_in,skiprows=1)
        tof=os.path.join(topath,mfile)
        if not os.path.exists(tof):
            
            saveswc(swc,tof,scale=25)


In [9]:
'''For motif_figs:
'''

image_src='motif_figs'
# 读取所有 PNG 图片
color=(255,0,0)
images = []
for file_name in sorted(os.listdir(image_src)):
    if file_name.endswith('.png'):
        fpath=os.path.join(image_src,file_name)
        fimage=Image.open(fpath).resize([600,600]).rotate(270)
        drawimage=ImageDraw.Draw(fimage)
        basename=file_name.split('.')[0]
        text=basename.split('-')[0]+': '+basename.split('-')[1]
        font=ImageFont.truetype('Arial Unicode.ttf', size=50)
        # text_width, text_height = drawimage.textsize(text, font)
        x = 150
        y = 30
        drawimage.text((x, y), text, font=font, fill=color)
        # break
        images.append(fimage)
images=np.array(images)
# fimage
# 将图片转换为视频
fps = 1  # 每秒钟30帧
filepath='./motif.mp4'
with imageio.get_writer(filepath, fps=fps) as video:
    for image in images:
        frame = image
        video.append_data(frame)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 600) to (608, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[swscaler @ 0x7fd213e6f000] Warning: data is not aligned! This can lead to a speed loss


In [33]:
# arbor
arbordir='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/SD/release/arbor/CCFv3_sep'
todir='/Users/jiangshengdian/Desktop/Daily/PhD_project/Platform/SD/release/arbor/CCFv3_25um_sep'
if not os.path.exists(todir):
    os.mkdir(todir)

for root,dirs,files in os.walk(arbordir):
    for mfile in files:
        if os.path.splitext(mfile)[1] != '.swc':
            continue
        topath=os.path.join(todir,mfile.split('_axon')[0])
        if not os.path.exists(topath):
            os.mkdir(topath)
        # break
        file_in=os.path.join(root,mfile)
        swc=readswc_arbor(file=file_in,skiprows=3)
        tof=os.path.join(topath,mfile)
        if not os.path.exists(tof):
            saveswc(swc.drop(['label'],axis=1),tof,scale=25)
        # break

In [52]:
'''For arbor_figs:
'''

all1876_feas_file='R1876_feas.csv'
all_1876_feas=pd.read_csv(all1876_feas_file,index_col=['name'])
image_src='arbor_figs'
toimage_src='arbor_figs_q10'
# 读取所有 PNG 图片
color=(255,0,0)
# images = []
for region in all_1876_feas['soma_region'].value_counts().keys().tolist():
    tregion=all_1876_feas[all_1876_feas['soma_region']==region].copy()
    for cell in tregion.index:
        file_name=cell+'.png'
        fpath=os.path.join(image_src,file_name)
        tofig=os.path.join(toimage_src,file_name)
        if os.path.exists(fpath) and not os.path.exists(tofig):
            fimage=Image.open(fpath).resize([600,600]).rotate(270)
            drawimage=ImageDraw.Draw(fimage)
            font=ImageFont.truetype('Arial Unicode.ttf', size=40)
            font1=ImageFont.truetype('Arial Unicode.ttf', size=38)
            drawimage.text((40, 20), cell, font=font, fill=(0,0,0))
            drawimage.text((50, 100), region, font=font1, fill=color)
            # break
            # images.append(fimage)
            fimage.save(tofig,quality=1)

# images=np.array(images)
# fimage


In [53]:
all1876_feas_file='R1876_feas.csv'
all_1876_feas=pd.read_csv(all1876_feas_file,index_col=['name'])
image_src='arbor_figs_q10'
# 读取所有 PNG 图片
color=(255,0,0)
images = []
for region in all_1876_feas['soma_region'].value_counts().keys().tolist():
    tregion=all_1876_feas[all_1876_feas['soma_region']==region].copy()
    for cell in tregion.index:
        file_name=cell+'.png'
        fpath=os.path.join(image_src,file_name)
        if os.path.exists(fpath):
            fimage=Image.open(fpath)
            # drawimage=ImageDraw.Draw(fimage)
            # text=region
            # font=ImageFont.truetype('Arial Unicode.ttf', size=38)
            # # text_width, text_height = drawimage.textsize(text, font)
            # x = 10
            # y = 50
            # drawimage.text((x, y), text, font=font, fill=color)
            # # break
            images.append(fimage)

images=np.array(images)

In [54]:
fps = 3
filepath='./arbor.mp4'
with imageio.get_writer(filepath, fps=fps) as video:
    for image in images:
        frame = image
        video.append_data(frame)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 600) to (608, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[swscaler @ 0x7fc441e7f000] Warning: data is not aligned! This can lead to a speed loss


In [64]:
'''Resize Fig data'''
main_figs='../static/images'
for i in np.arange(1,9):
    figpath=os.path.join(main_figs,'Fig'+str(i)+'.png')
    fimage=Image.open(figpath).resize([1500,1200])
    tofigpath=os.path.join(main_figs,'Fig'+str(i)+'_resized.png')
    fimage.save(tofigpath)
    # break